In [1]:
import torch
import numpy as np
import PIL

print(torch.cuda.is_available())

True


In [2]:
!nvidia-smi

Sat Jul 27 21:55:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 ...    Off | 00000000:01:00.0  On |                  N/A |
| N/A   52C    P5              22W / 125W |    174MiB / 16384MiB |     41%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms, models, datasets
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

In [4]:
DATA_MODES = ['train', 'val', 'test']
RESCALE_SIZE = 224
DEVICE = torch.device("cuda")

In [5]:
train_transforms = transforms.Compose([
    transforms.Resize((RESCALE_SIZE, RESCALE_SIZE)),
    #transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x / 255),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [6]:
train_dir = "./train/simpsons_dataset"

In [7]:
import math

image_datasets = datasets.ImageFolder(train_dir, train_transforms)

In [8]:
dataset_len = len(image_datasets)
p1 = (dataset_len / 100)
train_size = math.floor(p1 * 70)
test_size = dataset_len - train_size

In [9]:
train_dataset, val_dataset = torch.utils.data.random_split(image_datasets, [train_size, test_size])

In [10]:
len(train_dataset), len(val_dataset)

(14653, 6280)

In [11]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32,
    shuffle=True
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=128
)

loaders = {
    "train": train_dataloader,
    "valid": val_dataloader
}

In [12]:
model_resnet18 = models.resnet18(pretrained=True)
model_resnet18

/home/brotiger/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/brotiger/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
for param in model_resnet18.parameters():
    param.requires_grad = False

In [14]:
for param in model_resnet18.avgpool.parameters():
    param.requires_grad = True
    
for param in model_resnet18.layer4.parameters():
    param.requires_grad = True

for param in model_resnet18.layer3.parameters():
    param.requires_grad = True

for param in model_resnet18.layer2.parameters():
    param.requires_grad = True

In [15]:
num_classes = 42
model_resnet18.fc = nn.Sequential(
    nn.Sequential(
        nn.Linear(512, 512),
        nn.BatchNorm1d(512),
        nn.Dropout(0.5),
        nn.ReLU()
    ),
    nn.Sequential(
        nn.Linear(512, 512),
        nn.Linear(512, num_classes)
    )
) 

In [16]:
import torch.optim as optim

optimizer = optim.Adam(
    (
        {
            "params": model_resnet18.layer2.parameters(),
            "lr": 1e-5
        },
        {
            "params": model_resnet18.layer3.parameters(),
            "lr": 1e-4
        },
        {
            "params": model_resnet18.layer4.parameters(),
            "lr": 1e-3
        },
        {
            "params": model_resnet18.avgpool.parameters()
        },
        {
            "params": model_resnet18.fc.parameters()
        }
    ), lr=1e-2)
scheduler = optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.5)

In [17]:
!rm -rf logs

In [18]:
from catalyst import dl
        
runner = dl.SupervisedRunner()

In [19]:
runner.train(
    model=model_resnet18,
    engine=dl.GPUEngine("cuda"),
    optimizer=optimizer,
    criterion=nn.CrossEntropyLoss(),
    scheduler=scheduler,
    callbacks=[
        dl.CriterionCallback(input_key="logits", target_key="targets", metric_key="loss"),
        dl.BackwardCallback(metric_key="loss"),
        dl.OptimizerCallback(metric_key="loss"), 
        dl.AccuracyCallback(input_key="logits", target_key="targets"),
        dl.SchedulerCallback(),
        dl.PrecisionRecallF1SupportCallback(
            input_key="logits", target_key="targets", num_classes=num_classes, log_on_batch=False
        )
    ],
    loaders=loaders,
    num_epochs=30,
    verbose=True,
    logdir="logs/resnet18",
    load_best_on_end=True,
)

1/30 * Epoch (train):   0%|          | 0/458 [00:00<?, ?it/s]

train (1/30) accuracy01: 0.4448918309836864 | accuracy01/std: 0.14677202263423728 | f1/_macro: 0.1911559226572267 | f1/_micro: 0.44488683108055643 | f1/_weighted: 0.4279054927507017 | loss: 2.6606458742236385 | loss/mean: 2.6606458742236385 | loss/std: 1.3747384786155932 | lr: 1e-05 | momentum: 0.9 | precision/_macro: 0.19780175630811855 | precision/_micro: 0.4448918310243636 | precision/_weighted: 0.420551177039226 | recall/_macro: 0.19384639480145335 | recall/_micro: 0.4448918310243636 | recall/_weighted: 0.44489183102436364


/home/brotiger/.local/lib/python3.10/site-packages/accelerate/utils/dataclasses.py:480: FutureWarning: The `TPU` of `<enum 'DistributedType'>` is deprecated and will be removed in v1.0.0. Please use the `XLA` instead.
  warnings.warn(


1/30 * Epoch (valid):   0%|          | 0/50 [00:00<?, ?it/s]

valid (1/30) accuracy01: 0.637579617834395 | accuracy01/std: 0.039323259169996655 | f1/_macro: 0.27683675651641576 | f1/_micro: 0.6375746178736054 | f1/_weighted: 0.6118302934678506 | loss: 1.4510923236798328 | loss/mean: 1.4510923236798328 | loss/std: 0.1769923214046265 | lr: 1e-05 | momentum: 0.9 | precision/_macro: 0.306965687814668 | precision/_micro: 0.6375796178343949 | precision/_weighted: 0.6355023066597371 | recall/_macro: 0.2804407909988929 | recall/_micro: 0.6375796178343949 | recall/_weighted: 0.6375796178343949
* Epoch (1/30) lr: 1e-05 | momentum: 0.9


...

valid (29/30) accuracy01: 0.9194267515923567 | accuracy01/std: 0.02388290569719119 | f1/_macro: 0.7305164334287401 | f1/_micro: 0.9194217516195473 | f1/_weighted: 0.9221028081501816 | loss: 0.6668772015601967 | loss/mean: 0.6668772015601967 | loss/std: 0.3127477589247396 | lr: 3.125e-07 | momentum: 0.9 | precision/_macro: 0.7292908167072221 | precision/_micro: 0.9194267515923567 | precision/_weighted: 0.9271200185600691 | recall/_macro: 0.7514018947810511 | recall/_micro: 0.9194267515923567 | recall/_weighted: 0.9194267515923568
* Epoch (29/30) lr: 3.125e-07 | momentum: 0.9


30/30 * Epoch (train):   0%|          | 0/458 [00:00<?, ?it/s]

train (30/30) accuracy01: 0.997747901453627 | accuracy01/std: 0.008593127916584222 | f1/_macro: 0.9816266402281535 | f1/_micro: 0.9977429014786835 | f1/_weighted: 0.9977601710393026 | loss: 0.007360248117538803 | loss/mean: 0.007360248117538803 | loss/std: 0.023250174255772575 | lr: 3.125e-07 | momentum: 0.9 | precision/_macro: 0.9788412855710024 | precision/_micro: 0.9977479014536272 | precision/_weighted: 0.9978329735712502 | recall/_macro: 0.9855915819848254 | recall/_micro: 0.9977479014536272 | recall/_weighted: 0.9977479014536274


30/30 * Epoch (valid):   0%|          | 0/50 [00:00<?, ?it/s]

valid (30/30) accuracy01: 0.9195859872611466 | accuracy01/std: 0.02334563948756022 | f1/_macro: 0.7371632765368668 | f1/_micro: 0.9195809872883325 | f1/_weighted: 0.9222059463966924 | loss: 0.6821966795974473 | loss/mean: 0.6821966795974473 | loss/std: 0.3267253254208207 | lr: 3.125e-07 | momentum: 0.9 | precision/_macro: 0.747491542908389 | precision/_micro: 0.9195859872611465 | precision/_weighted: 0.9272567892566954 | recall/_macro: 0.7435132081459291 | recall/_micro: 0.9195859872611465 | recall/_weighted: 0.9195859872611466
* Epoch (30/30) lr: 1.5625e-07 | momentum: 0.9
Top models:
logs/resnet18/checkpoints/model.0030.pth	30.0000


In [20]:
%load_ext tensorboard
%tensorboard --logdir logs

In [21]:
import os

class TestDataset(Dataset):
    def __init__(self, root, transforms):
        super().__init__()
        self.files = sorted(list(Path(root).rglob('*.jpg')), key=self.extract_image_number)
        self.transforms = transforms

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        x = self.load_sample(self.files[index])
        return self.transforms(x), 0

    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image

    def extract_image_number(self, path):
        filename = os.path.basename(path)
        return int(filename[3:-4])

In [22]:
test_transforms = transforms.Compose([
    transforms.Resize((RESCALE_SIZE, RESCALE_SIZE)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x / 255),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [23]:
test_dir = "./testset/testset"
test_dataset = TestDataset(test_dir, test_transforms)

In [24]:
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=128
)

In [25]:
result = []

for prediction in runner.predict_loader(engine=dl.GPUEngine("cuda"), loader=test_dataloader):
    result += prediction['logits'].detach().cpu().numpy().argmax(1).tolist()

In [26]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(image_datasets.classes)

LabelEncoder()

In [27]:
import pandas as pd

sample_submission_path = 'sample_submission.csv'
submission = pd.read_csv(sample_submission_path)
submission.head()

,Id,Expected
0,img0.jpg,bart_simpson
1,img1.jpg,bart_simpson
2,img2.jpg,bart_simpson
3,img3.jpg,bart_simpson
4,img4.jpg,bart_simpson


In [28]:
submission['Expected'] = label_encoder.inverse_transform(result)
submission

,Id,Expected
0,img0.jpg,nelson_muntz
1,img1.jpg,bart_simpson
2,img2.jpg,mayor_quimby
3,img3.jpg,nelson_muntz
4,img4.jpg,lisa_simpson
...,...,...
986,img986.jpg,sideshow_bob
987,img987.jpg,nelson_muntz
988,img988.jpg,ned_flanders
989,img989.jpg,charles_montgomery_burns


In [29]:
submission_path = 'submission.csv'
submission.to_csv(submission_path, index=None)